
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/78_RAG_Evaluation_RAGAS.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# ⚖️ RAG Evaluation: Sędzia AI (LLM-as-a-Judge)

W klasycznym ML mamy `Accuracy` (czy 0=0?). W generatywnym AI odpowiedź jest tekstem. Nie da się jej porównać "znak po znaku".

**Paradygmat LLM-as-a-Judge:**
Używamy jednego modelu AI (Sędziego), żeby oceniał drugi model (Studenta).

Zbudujemy system oceny **Faithfulness (Wierności)**:
1.  **Krok 1:** Wyciągnij z odpowiedzi Studenta wszystkie twierdzenia (fakty).
2.  **Krok 2:** Sprawdź każdy fakt w Kontekście (źródłach).
3.  **Krok 3:** Jeśli fakt nie ma pokrycia w źródłach -> **Halucynacja**.
4.  **Wynik:** Liczba faktów potwierdzonych / Liczba wszystkich faktów.

To jest dokładnie to, co robi biblioteka **RAGAS** pod maską.

In [1]:
import pandas as pd
import json

# 1. DANE DO OCENY (Symulacja logów z Twojego RAG-a)
# Mamy 3 przypadki:
# A: Idealna odpowiedź.
# B: Halucynacja (zmyśla fakty spoza kontekstu).
# C: Nie na temat (Unikanie odpowiedzi).

rag_logs = [
    {
        "id": "Case_A_Good",
        "question": "Jaka jest stolica Francji?",
        "context": "Francja to kraj w Europie. Jej stolicą jest Paryż.",
        "answer": "Stolicą Francji jest Paryż."
    },
    {
        "id": "Case_B_Hallucination",
        "question": "Kto wygrał mundial w 2022?",
        "context": "Mundial 2022 odbył się w Katarze. Brała w nim udział Polska.",
        "answer": "Mundial w 2022 wygrała Argentyna." 
        # To prawda w życiu, ale FAŁSZ w kontekście RAG! 
        # Model użył wiedzy z głowy, a nie z dokumentu. To błąd Faithfulness.
    },
    {
        "id": "Case_C_Irrelevant",
        "question": "Jak zresetować hasło?",
        "context": "Aby zresetować hasło, kliknij w link w mailu.",
        "answer": "Hasła są ważne dla bezpieczeństwa."
        # Prawda, ale nie odpowiada na pytanie. Błąd Relevance.
    }
]

df = pd.DataFrame(rag_logs)
print("--- LOGI Z RAG ---")
display(df)

--- LOGI Z RAG ---


,id,question,context,answer
0,Case_A_Good,Jaka jest stolica Francji?,Francja to kraj w Europie. Jej stolicą jest Pa...,Stolicą Francji jest Paryż.
1,Case_B_Hallucination,Kto wygrał mundial w 2022?,Mundial 2022 odbył się w Katarze. Brała w nim ...,Mundial w 2022 wygrała Argentyna.
2,Case_C_Irrelevant,Jak zresetować hasło?,"Aby zresetować hasło, kliknij w link w mailu.",Hasła są ważne dla bezpieczeństwa.


## Budowa Sędziego (The Judge)

Nie będziemy tu podinać płatnego API OpenAI.
Zapiszemy **Prompty Sędziowskie**, które normalnie wysłałbyś do GPT-4.
Zasymulujemy działanie Sędziego, żebyś zrozumiał algorytm oceniania.

In [2]:
class RAGEvaluator:
    def __init__(self):
        pass
    
    def create_faithfulness_prompt(self, context, answer):
        """
        Prompt sprawdzający, czy model nie kłamie.
        """
        return f"""
        Jesteś Sędzią weryfikującym fakty.
        
        ZADANIE:
        1. Rozbij 'Odpowiedź' na listę pojedynczych twierdzeń.
        2. Dla każdego twierdzenia sprawdź, czy wynika ono logicznie z 'Kontekstu'.
        3. Jeśli wynika -> TAK. Jeśli nie ma go w kontekście -> NIE.
        
        Kontekst: "{context}"
        Odpowiedź: "{answer}"
        
        Format wyjściowy (JSON):
        {{
            "statements": ["twierdzenie 1", "twierdzenie 2"],
            "verdict": ["TAK", "NIE"],
            "score": (liczba TAK / wszystkie)
        }}
        """

    def create_relevance_prompt(self, question, answer):
        """
        Prompt sprawdzający, czy model odpowiedział na temat.
        """
        return f"""
        Jesteś Sędzią oceniającym jakość rozmowy.
        
        ZADANIE:
        Oceń, czy 'Odpowiedź' jest użyteczna i bezpośrednio odnosi się do 'Pytania'.
        Ignoruj poprawność faktów (to sprawdzamy gdzie indziej). Skup się na intencji.
        
        Pytanie: "{question}"
        Odpowiedź: "{answer}"
        
        Wynik: 0 (Kompletnie nie na temat) do 1 (Idealna odpowiedź).
        """

evaluator = RAGEvaluator()
print("Sędzia gotowy. Możemy generować instrukcje dla GPT-4.")

Sędzia gotowy. Możemy generować instrukcje dla GPT-4.


## Symulacja Oceny (Case B: Halucynacja)

Spójrzmy na przypadek B:
*   Context: "Mundial był w Katarze."
*   Answer: "Wygrała Argentyna."

Dla człowieka to prawda.
Dla systemu RAG to **Błąd Krytyczny**. Jeśli w dokumentach firmowych nie ma info o Argentynie, model nie ma prawa o tym pisać. To się nazywa **Data Leakage from Pre-training**.

In [3]:
case = rag_logs[1] # Case B

print(f"--- ANALIZA PRZYPADKU: {case['id']} ---")
print(f"Pytanie: {case['question']}")
print(f"Kontekst: {case['context']}")
print(f"Odpowiedź Modelu: {case['answer']}")

print("\n--- CO WIDZI SĘDZIA (PROMPT) ---")
prompt = evaluator.create_faithfulness_prompt(case['context'], case['answer'])
print(prompt)

print("\n--- SYMULOWANA ODPOWIEDŹ SĘDZIEGO (GPT-4) ---")
# To jest to, co zwróciłby model:
simulated_verdict = {
    "statements": ["Mundial w 2022 wygrała Argentyna"],
    "verdict": ["NIE"],
    "reasoning": "W podanym kontekście nie ma informacji o zwycięzcy. Jest tylko info o lokalizacji.",
    "score": 0.0
}
print(json.dumps(simulated_verdict, indent=2, ensure_ascii=False))

print("-" * 30)
if simulated_verdict['score'] < 1.0:
    print("🚨 FAITHFULNESS FAIL: Model użył wiedzy spoza dokumentu!")

--- ANALIZA PRZYPADKU: Case_B_Hallucination ---
Pytanie: Kto wygrał mundial w 2022?
Kontekst: Mundial 2022 odbył się w Katarze. Brała w nim udział Polska.
Odpowiedź Modelu: Mundial w 2022 wygrała Argentyna.

--- CO WIDZI SĘDZIA (PROMPT) ---

        Jesteś Sędzią weryfikującym fakty.

        ZADANIE:
        1. Rozbij 'Odpowiedź' na listę pojedynczych twierdzeń.
        2. Dla każdego twierdzenia sprawdź, czy wynika ono logicznie z 'Kontekstu'.
        3. Jeśli wynika -> TAK. Jeśli nie ma go w kontekście -> NIE.

        Kontekst: "Mundial 2022 odbył się w Katarze. Brała w nim udział Polska."
        Odpowiedź: "Mundial w 2022 wygrała Argentyna."

        Format wyjściowy (JSON):
        {
            "statements": ["twierdzenie 1", "twierdzenie 2"],
            "verdict": ["TAK", "NIE"],
            "score": (liczba TAK / wszystkie)
        }
        

--- SYMULOWANA ODPOWIEDŹ SĘDZIEGO (GPT-4) ---
{
  "statements": [
    "Mundial w 2022 wygrała Argentyna"
  ],
  "verdict": [
    "NIE

## Symulacja Oceny (Case C: Relevance)

Spójrzmy na przypadek C:
*   Pytanie: "Jak zresetować hasło?"
*   Odpowiedź: "Hasła są ważne."

Model powiedział prawdę, oparł się na swojej wiedzy ogólnej, ale... nie pomógł użytkownikowi.

In [4]:
case = rag_logs[2] # Case C

print(f"--- ANALIZA PRZYPADKU: {case['id']} ---")
prompt = evaluator.create_relevance_prompt(case['question'], case['answer'])
print(prompt)

print("\n--- SYMULOWANA ODPOWIEDŹ SĘDZIEGO ---")
simulated_verdict_relevance = {
    "score": 0.2,
    "reasoning": "Użytkownik pytał o instrukcję (JAK), model podał opinię/fakt ogólny. Nie rozwiązuje problemu."
}
print(json.dumps(simulated_verdict_relevance, indent=2, ensure_ascii=False))

--- ANALIZA PRZYPADKU: Case_C_Irrelevant ---

        Jesteś Sędzią oceniającym jakość rozmowy.

        ZADANIE:
        Oceń, czy 'Odpowiedź' jest użyteczna i bezpośrednio odnosi się do 'Pytania'.
        Ignoruj poprawność faktów (to sprawdzamy gdzie indziej). Skup się na intencji.

        Pytanie: "Jak zresetować hasło?"
        Odpowiedź: "Hasła są ważne dla bezpieczeństwa."

        Wynik: 0 (Kompletnie nie na temat) do 1 (Idealna odpowiedź).
        

--- SYMULOWANA ODPOWIEDŹ SĘDZIEGO ---
{
  "score": 0.2,
  "reasoning": "Użytkownik pytał o instrukcję (JAK), model podał opinię/fakt ogólny. Nie rozwiązuje problemu."
}


## 🧠 Podsumowanie: Trust but Verify

W inżynierii LLM nie ufamy modelom na słowo. Budujemy **Testy Jednostkowe** oparte na innym LLM.

**Jak to wdrożyć w firmie?**
1.  Zbierasz 50 par (Pytanie, Idealna Odpowiedź) – tzw. **Golden Dataset**.
2.  Zmieniasz kod swojego RAG-a (np. zmieniasz Chunk Size).
3.  Puszczasz te 50 pytań przez nowy system.
4.  Używasz GPT-4 jako Sędziego do oceny Faithfulness i Relevance.
5.  Jeśli średni wynik spadł – nie wdrażasz zmian.

Bez tego kręcisz się w kółko ("Wydaje mi się, że teraz działa lepiej").